In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np

In [2]:
image_size = [224, 224]
vgg = VGG16(input_shape = image_size + [3], weights = 'imagenet', include_top =  False)

In [3]:
for layer in vgg.layers:
    layer.trainable = False

In [4]:
from glob import glob
folders = glob('D:/ST/trainData/Tomato/*')
folders

['D:/ST/trainData/Tomato\\Tomato_leafMold',
 'D:/ST/trainData/Tomato\\Tomato_leafSpot',
 'D:/ST/trainData/Tomato\\Tomato_nomal',
 'D:/ST/trainData/Tomato\\Tomato_trifolii',
 'D:/ST/trainData/Tomato\\Tomato_yellowLeafCurlVirus']

In [5]:
outlayer = Flatten()(vgg.output)
outlayer = Dense(4096, activation='relu',name='fc1')(outlayer)
outlayer = Dropout(0.5)(outlayer)
outlayer = Dense(4096, activation='relu',name='fc2')(outlayer)
prediction = Dense(len(folders), activation = 'softmax')(outlayer)
model = Model(inputs = vgg.input, outputs = prediction)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
#이미지 전처리 무작위 변형
train_data_gen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2 \
                                    , zoom_range = 0.2, horizontal_flip = True)

In [10]:
train_set = train_data_gen.flow_from_directory('D:/ST/trainData/Tomato/' \
                                               , target_size = (224,224) \
                                               , shuffle = True \
                                               , batch_size = 16, class_mode = 'categorical')

Found 1668 images belonging to 5 classes.


In [11]:
test_data_gen = ImageDataGenerator(rescale = 1./255)

In [12]:
test_set = test_data_gen.flow_from_directory('D:/ST/trainData/Tomato_val/' \
                                             , target_size = (224,224) \
                                             , shuffle = True \
                                             , batch_size = 16, class_mode = 'categorical')

Found 409 images belonging to 5 classes.


In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 

path_checkpoint = "Tomato_model_checkpoint.h5"
es_callback = EarlyStopping(monitor="val_loss", min_delta=0, patience=5)
modelckpt_callback = ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

with tf.device("/device:CPU:0"):
    mod = model.fit_generator(train_set,epochs=10, steps_per_epoch=len(train_set) \
                    ,callbacks=[es_callback, modelckpt_callback] \
                    ,validation_data=test_set, validation_steps=len(test_set))

C:\Users\USER-PC\AppData\Local\Temp\ipykernel_14708\1667835563.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  mod = model.fit_generator(train_set,epochs=10, steps_per_epoch=len(train_set) \


Epoch 1/10
105/105 [==============================] - ETA: 0s - loss: 2.8379 - accuracy: 0.5845
Epoch 00001: val_loss improved from inf to 1.53134, saving model to Tomato_model_checkpoint.h5
105/105 [==============================] - 547s 5s/step - loss: 2.8379 - accuracy: 0.5845 - val_loss: 1.5313 - val_accuracy: 0.4010
Epoch 2/10
105/105 [==============================] - ETA: 0s - loss: 0.7916 - accuracy: 0.7002
Epoch 00002: val_loss improved from 1.53134 to 1.23767, saving model to Tomato_model_checkpoint.h5
105/105 [==============================] - 484s 5s/step - loss: 0.7916 - accuracy: 0.7002 - val_loss: 1.2377 - val_accuracy: 0.4645
Epoch 3/10
105/105 [==============================] - ETA: 0s - loss: 0.7128 - accuracy: 0.7218
Epoch 00003: val_loss did not improve from 1.23767
105/105 [==============================] - 524s 5s/step - loss: 0.7128 - accuracy: 0.7218 - val_loss: 1.5036 - val_accuracy: 0.5086
Epoch 4/10
105/105 [==============================] - ETA: 0s - loss: 0

In [13]:
"""from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 

path_checkpoint = "Tomato_model_checkpoint.h5"
es_callback = EarlyStopping(monitor="val_loss", min_delta=0, patience=5)
modelckpt_callback = ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

with tf.device("/device:CPU:0"):
    mod = model.fit(train_set,epochs=10 \
                    ,callbacks=[es_callback, modelckpt_callback] \
                    ,validation_data=test_set)"""

Epoch 1/5
53/53 [==============================] - ETA: 0s - loss: 7.0346 - accuracy: 0.5108 
Epoch 00001: val_loss improved from inf to 1.74667, saving model to Tomato_model_checkpoint.h5
53/53 [==============================] - 689s 13s/step - loss: 7.0346 - accuracy: 0.5108 - val_loss: 1.7467 - val_accuracy: 0.4401
Epoch 2/5
53/53 [==============================] - ETA: 0s - loss: 0.7841 - accuracy: 0.6829
Epoch 00002: val_loss improved from 1.74667 to 1.33623, saving model to Tomato_model_checkpoint.h5
53/53 [==============================] - 449s 9s/step - loss: 0.7841 - accuracy: 0.6829 - val_loss: 1.3362 - val_accuracy: 0.4621
Epoch 3/5
53/53 [==============================] - ETA: 0s - loss: 0.6809 - accuracy: 0.7338
Epoch 00003: val_loss improved from 1.33623 to 1.14839, saving model to Tomato_model_checkpoint.h5
53/53 [==============================] - 471s 9s/step - loss: 0.6809 - accuracy: 0.7338 - val_loss: 1.1484 - val_accuracy: 0.4866
Epoch 4/5
53/53 [===================

In [14]:
model.save("Model_5class_test2")

INFO:tensorflow:Assets written to: Model_5class_test2\assets
